In [ ]:
import json
import traceback
import psycopg2
from fuzzywuzzy import process


list_especial_names = {
    'V. Junior': 'Vinicius Jr.',
    'Y. E. Nesyri': 'En-Nesyri',
    'Y. Gasiorowski': '',
    'N. Fernandez': '',
    'V. Fernandez': '',
    'B. Espinosa': '',
    'R. Fernandez': '',
    'G. Moreno': 'A. Moreno',
    'A. Munoz': 'I. Munoz',
    'E. Avila': 'C. Avila',
    'P. Espino': 'Espino',
    'A. Vinicius': 'Abner',
    'K. Fernandes': 'Kaiky',
    "F. Gimenez": "Fali",
    "L. Vinicius": "Lazaro",
}

list_2_names = {
    "Sandro": "S. Ramirez",
    "R. G. D. Haro": "R. Garcia",  # Osasuna
    "J. C. Martin": "J. Carlos",
    "I. Garcia": "I. G. D. Albeniz",
    "O. Mfulu": "N. Mfulu"
}

list_3_names = {
    "N. Fernandez": "Nacho",
    'A. Lemos': 'Lemos. S',
    'I. Garcia': 'Imanol',
    'C. Ozkacar': 'Cenk',
    'U. Gomez': 'Unai. G',
    'B. Zaragoza': 'Bryan',
    'I. Palazon': 'Isi',
    'A. Srloth': 'Sorloth',
    'G. Alvarez': 'Gaston',
    'S. Niguez': 'Saul',
    'S. Omorodion': 'Samu',
    'R. Marti': 'Roger',
    'F. Lopez': 'Fermin',
    'C. Mosquera': 'Cristhian',
    'M. Depay': 'Memphis',
    'M. E. Haddadi': 'Munir',
    'B. Diaz': 'Brahim',
    'A. Ferreira': 'Andre. F.',
    'Y. Berchiche': 'Yuri',
    'A. Ezzalzouli': 'Abde',
    'A. Prats': 'Abdon',
    'R. D. Tomas': 'Rdt',
    'M. Park': 'Marvin',
    'I. Carcelen': 'Iza',
    'A. Elustondo': 'Aritz',
    'R. Mandava': 'Reinildo',
    'D. Lopy': 'Dion',
    'A. Ratiu': 'Andrei',
    'R. G. D. Haro': 'Raul',  # Osasuna
    'M. Pascual': 'Martin',
    'R. Fernandez': 'Raul. F',
    'S. Clua': 'Selvi',
    'B. Ramirez': 'Benito. J',
    'A. Ndiaye': 'Amath',
    'V. Fernandez': 'Valery',
    'Y. Alvarez': 'Yeray',
    'M. Milovanovic': 'Marezi',
    'B. Espinosa': 'Bernardo',
    'B. B. Diaz': 'B. Brereton',
    'M. Diaz': 'Mariano',
}


# Función para insertar equipos únicos en la tabla team
def insert_teams(conn, players_data):
    cur = conn.cursor()
    # Suponiendo que id_team es SERIAL y se autoincrementa, y que name es UNIQUE.
    inserted_teams = set()  # Para evitar intentos de inserción duplicados
    query_team = "INSERT INTO team (name) VALUES (%s) ON CONFLICT (name) DO NOTHING;"
    for player in players_data:
        team_name = player['team']
        if team_name not in inserted_teams:  # Revisar si el equipo ya fue procesado
            cur.execute(query_team, (team_name,))
            conn.commit()
            inserted_teams.add(team_name)
    cur.close()

# Función para insertar datos de jugadores en la base de datos, actualizada para incluir team_id y team_name
def insert_player_data(conn, players_data):
    cur = conn.cursor()
    # Asume que has añadido una columna 'team_name' a tu tabla 'player'
    query_player = """
    INSERT INTO player (name, foot, team_id, team, short_name_position, age, shirt_number, market_value_mister_fantasy, nationality, image_url)
    SELECT %s, %s, t.id_team, t.name, %s, %s, %s, %s, %s, %s
    FROM team t
    WHERE t.name = %s;
    """
    for player in players_data:
        try:
            cur.execute(query_player, (
                player.get('name'),
                player.get('foot'),
                player.get('shortNamePosition'),
                player.get('age'),
                player.get('shirtNumber'),
                player.get('marketValue'),
                player.get('nationality'),
                player.get('image'),  # Asegúrate de que este campo coincida con tu columna en la base de datos.
                player.get('team'),  # Último parámetro para WHERE t.name
            ))
            conn.commit()  # Comprometer la transacción si no hay errores.
        except Exception as e:
            print(f"Error al insertar el jugador {player.get('name')}: {e}")
            conn.rollback()  # Revertir la transacción si hay un error.
    cur.close()

try:
    # Define conn aquí para asegurarte de que está en el scope adecuado
    conn = psycopg2.connect(
        database="DraftKingsBD",
        user="postgres",
        password="admin",
        host='localhost',
        port='5432'  # Ensure port is a string if specified directly in the connect call
    )
    print("Connection established:", conn)
    # Asegúrate de que la ruta al archivo JSON es correcta y accesible
    with open('TransferMarket.json', 'r', encoding='utf-8') as file:
        players_data = json.load(file)['Players']
    insert_teams(conn, players_data)  # Primero insertamos los equipos
    insert_player_data(conn, players_data)  # Luego insertamos los jugadores
except psycopg2.DatabaseError as db_err:
    print(f"Database error: {db_err}")
    traceback.print_exc()
except Exception as e:
    print(f"An error occurred: {e}")
    traceback.print_exc()
finally:
    if conn is not None:
        conn.close()